In [6]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import numpy as np

In [7]:
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [9]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [10]:
# 模型测试
message= "命名实体识别"
label = torch.tensor([0,1,0,2,5,4])

model_input = tokenizer([message], return_tensors='pt')
result = model(**model_input)

print(result.loss)
print(result.logits)

None
tensor([[[ 0.3389, -0.1038,  0.7628, -0.0923,  0.2169,  0.1954,  0.2398],
         [ 0.8772,  0.0736,  0.5088,  0.4887,  0.2392,  0.3507,  0.0748],
         [-0.0131,  0.2049, -0.5644, -0.2755, -0.7436,  0.0763,  0.3688],
         [ 0.4493,  0.1104, -0.8357,  0.0921, -0.9937,  0.2044,  0.2632],
         [ 0.2270,  0.2171, -0.6591, -0.2837, -0.3419,  0.0233,  0.1571],
         [-0.1314, -0.0120, -0.2281, -0.2134, -0.2069, -0.1293,  0.5736],
         [-0.3765, -0.4535, -0.5427, -0.4635, -0.6766, -0.2555,  0.9607],
         [ 0.5152,  0.0869, -0.5296,  0.0581,  0.2645, -0.0101, -0.0743]]],
       grad_fn=<ViewBackward0>)


In [11]:
# 加载hf中dataset
ds = load_dataset('doushabao4766/msra_ner_k_V3')
ds

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [28]:
ds["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'knowledge'],
    num_rows: 45001
})

## 实体映射数据集词典准备

In [12]:
# entity_index
entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
           'company', 'scene', 'book', 'organization', 'government'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}


In [15]:
entity_index

{'O': 0,
 'company': 1,
 'game': 2,
 'government': 3,
 'scene': 4,
 'book': 5,
 'name': 6,
 'organization': 7,
 'address': 8,
 'position': 9,
 'movie': 10}

In [16]:
tags

['O',
 'B-COMPANY',
 'I-COMPANY',
 'B-GAME',
 'I-GAME',
 'B-GOVERNMENT',
 'I-GOVERNMENT',
 'B-SCENE',
 'I-SCENE',
 'B-BOOK',
 'I-BOOK',
 'B-NAME',
 'I-NAME',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-ADDRESS',
 'I-ADDRESS',
 'B-POSITION',
 'I-POSITION',
 'B-MOVIE',
 'I-MOVIE']

In [13]:
def entity_tags_proc(item):
    # item即是dataset中记录
    # text_len = len(item['text'])  # 根据文本长度生成tags列表
    # tags = [0] * text_len    # 初始值为‘O’
    # 遍历实体列表，所有实体类别标记填入tags
    tags = item['ents']
    # for ent in entites:
    #     indices = ent['indices']  # 实体索引
    #     label = ent['label']   # 实体名
    #     tags[indices[0]] = entity_index[label] * 2 - 1
    #     for idx in indices[1:]:
    #         tags[idx] = entity_index[label] * 2
    return {'ent_tag': tags}

# 使用自定义回调函数处理数据集记录
ds1 = ds.map(entity_tags_proc)

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

KeyError: 'ents'

In [30]:
# 训练集
for row in ds['train']:
    print(row)
    print(row['tokens'])
    # print(row['tokens'])
    # print(row['tokens'])
    # print(row['ner_tags'])
    # print(row['ent_tag'])
    
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': ''}
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']


In [20]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'ent_tag'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'ent_tag'],
        num_rows: 3443
    })
})

中文bert分词在日期时间和英文转换token过程中，出现合并。影响ner标注准确性。

In [14]:
token_index = tokenizer.encode('2000年2月add', add_special_tokens=False)
print(token_index)
tokens = tokenizer.decode(token_index)
print(tokens)

[8202, 2399, 123, 3299, 10253]
2000 年 2 月 add


In [15]:
input_data = tokenizer(['2000年2月add'], add_special_tokens=False, truncation=True)
print(input_data)

input_data.word_ids(0) # 返回批次中指定token对应原始文本的索引映射

{'input_ids': [[8202, 2399, 123, 3299, 10253]], 'token_type_ids': [[0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


[0, 1, 2, 3, 4]

In [16]:
def data_input_proc(item):
    # 处理批处理数据（batched=True时item是字典，每个值都是列表）
    texts = []
    for tokens in item['tokens']:
        if len(tokens) == 0:
            texts.append('')
            continue
        # 确保每个样本的token列表被正确转换为字符串
        if isinstance(tokens[0], list):
            tokens = [t for sublist in tokens for t in sublist]
        texts.append(''.join(tokens))
    input_data = tokenizer(texts, truncation=True, add_special_tokens=False, max_length=512)
    
    # 获取word_ids映射（批处理）
    # word_ids_batch = [input_data.word_ids(batch_index=i) for i in range(len(texts))]
    
    # 调整标签序列
    adjust_labels = []
    # 上一步骤生成ner_tag中索引和token对齐
    for k in range(len(input_data['input_ids'])):
        # 每条记录token对应word_ids
        word_ids = input_data.word_ids(k)
        # 批次ner_tag长度和token长度对齐
        tags = item['ner_tags'][k]
        
        adjusted_label_ids = []
        i, prev_wid = -1,-1
        for wid in word_ids:
            if (wid != prev_wid):   #  word_ids [1,1,1,2,3,4,5] -> [0,1,2,3,4,5,6]
                i += 1 # token对应检索位置+1
                prev_wid = wid
            adjusted_label_ids.append(tags[i])
        adjust_labels.append(adjusted_label_ids)                
    # 修正后label添加到input_data
    input_data['labels'] = adjust_labels
    return input_data

# 应用处理函数
ds2 = ds.map(data_input_proc, batched=True)

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [17]:
ds2

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})

In [18]:
# 训练集
for row in ds2['train']:
    print(row)
    
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': '', 'input_ids': [2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [19]:
# 训练集
for row in ds2['train']:
    print(row['id'])
    print(row['tokens'])
    print(row['ner_tags'])
    print(row['input_ids'])
    # print(row['token_type_ids'])
    print(row['attention_mask'])
    print(row['labels'])
    print()
    print(len(row['tokens']))
    print(len(row['ner_tags']))
    print(len(row['input_ids']))
    # print(row['token_type_ids'])
    print(len(row['attention_mask']))
    print(len(row['labels']))
    break

0
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5

In [20]:
# 记录转换为pytorch
ds2.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# ds_new = ds2.with_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [21]:
for item in ds2['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


## 模型训练
### TrainingArguments


In [27]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

### Model

In [23]:
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=21,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Trainer

In [24]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results


In [28]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

trainer = Trainer(
    model,
    args,
    train_dataset=ds2['train'],
    eval_dataset=ds2['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

模型训练

In [29]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Address,Game,Position,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.086700,0.044454,"{'precision': 0.8665943600867679, 'recall': 0.8419388830347735, 'f1': 0.854088722608231, 'number': 2847}","{'precision': 0.7631384159881569, 'recall': 0.7816527672479151, 'f1': 0.7722846441947565, 'number': 1319}","{'precision': 0.8803475935828877, 'recall': 0.873921698739217, 'f1': 0.8771228771228772, 'number': 1507}",0.845359,0.836418,0.840865,0.985045
2,0.043700,0.044853,"{'precision': 0.8779685787358421, 'recall': 0.8440463645943098, 'f1': 0.86067335243553, 'number': 2847}","{'precision': 0.821319018404908, 'recall': 0.8119787717968158, 'f1': 0.8166221883339687, 'number': 1319}","{'precision': 0.8977572559366754, 'recall': 0.9031187790311878, 'f1': 0.9004300363876943, 'number': 1507}",0.870074,0.852283,0.861086,0.986635
3,0.021800,0.044692,"{'precision': 0.8959308606409795, 'recall': 0.8739023533544081, 'f1': 0.8847795163584636, 'number': 2847}","{'precision': 0.8123167155425219, 'recall': 0.8400303260045489, 'f1': 0.8259411106969808, 'number': 1319}","{'precision': 0.9081967213114754, 'recall': 0.9190444591904446, 'f1': 0.9135883905013195, 'number': 1507}",0.879103,0.878019,0.878561,0.987760


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Trainer is attempting to log a value of "{'precision': 0.8665943600867679, 'recall': 0.8419388830347735, 'f1': 0.854088722608231, 'number': 2847}" of type <class 'dict'> for key "eval/ADDRESS" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7631384159881569, 'recall': 0.7816527672479151, 'f1': 0.7722846441947565, 'number': 1319}" of type <class 'dict'> for key "eval/GAME" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8803475935828877, 'recall': 0.873921698739217, 'f1': 0.8771228771228772, 'number': 1507}" of type <class 'dict'> for key "eval/POSITION" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was as

TrainOutput(global_step=2112, training_loss=0.0441932672578277, metrics={'train_runtime': 2298.9743, 'train_samples_per_second': 58.723, 'train_steps_per_second': 0.919, 'total_flos': 1.1735178399679266e+16, 'train_loss': 0.0441932672578277, 'epoch': 3.0})

模型推理

In [30]:
result = trainer.predict(ds2['test'])

In [32]:
print(ds['test'][10]['tokens'])
print(ds2['test'][10]['labels'])
print(result.label_ids[10])

['在', '跨', '世', '纪', '的', '征', '途', '上', '，', '在', '中', '国', '共', '产', '党', '领', '导', '下', '，', '我', '们', '要', '努', '力', '实', '现', '包', '括', '各', '民', '主', '党', '派', '、', '各', '人', '民', '团', '体', '、', '无', '党', '派', '人', '士', '在', '内', '的', '全', '体', '中', '国', '人', '民', '的', '大', '团', '结', '，', '实', '现', '包', '括', '大', '陆', '同', '胞', '、', '台', '港', '澳', '同', '胞', '和', '海', '外', '侨', '胞', '在', '内', '的', '所', '有', '爱', '国', '的', '中', '华', '儿', '女', '的', '大', '团', '结', '，', '从', '而', '战', '胜', '各', '种', '艰', '难', '险', '阻', '，', '实', '现', '跨', '世', '纪', '的', '宏', '伟', '蓝', '图', '。']
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 5, 5, 5, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[   0    0    0    0  

In [33]:
 [tags[p] for p,l in zip(result.label_ids[10],ds2['test'][10]['labels'])]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GAME',
 'I-GAME',
 'I-GAME',
 'I-GAME',
 'I-GAME',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'B-ADDRESS',
 'B-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [34]:
[tags[l] for p,l in zip(result.label_ids[10],ds2['test'][10]['labels'])]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GAME',
 'I-GAME',
 'I-GAME',
 'I-GAME',
 'I-GAME',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'B-ADDRESS',
 'B-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADDRESS',
 'I-ADDRESS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']